In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import os
from natsort import os_sorted
%matplotlib auto
pics_list = glob.glob("test-rqd/*.jpg")
pics_list=os_sorted(pics_list)

Using matplotlib backend: Qt5Agg


In [413]:
label_data = []
for pic in pics_list:
    img = cv2.imread(pic)
    width = 440
    height = 440
    
    dim = (width, height)
    lower = np.array([210,0,210])  #-- Lower range --
    upper = np.array([255,150,255])  #-- Upper range --
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    resized = resized[235:-10,:]

    # Pink mask
    lower = np.array([170,0,170])  #-- Lower range --
    upper = np.array([255,150,255])  #-- Upper range --
    pink_mask = cv2.inRange(resized, lower, upper)
#     cv2.imshow('Pink mask',pink_mask)


    # purple mask
    lower = np.array([0,0,0])  #-- Lower range --
    upper = np.array([255,5,255])  #-- Upper range --
    purple_mask = cv2.inRange(resized, lower, upper)
#     cv2.imshow(' purple mask',purple_mask)

    # combine masks

    combined_mask = cv2.bitwise_or(pink_mask, purple_mask, mask = None)
#     cv2.imshow('combinedmask',combined_mask)
#     cv2.waitKey(1)





    contours, hierarchy = cv2.findContours(image=combined_mask, mode=cv2.RETR_EXTERNAL   , method=cv2.CHAIN_APPROX_NONE )
   
    
    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)
    #         cv2.rectangle(resized, (x,y), (x+w,y+h), (0, 255, 0), 2)
        if 15<h and w>2:
            cv2.rectangle(resized, (x,y), (x+w,y+h), (0, 0, 0), -1)
            label_data.append([pic.split('\\')[1] , 'wood',x,y,w,h,resized.shape[0],resized.shape[1]])
#     cv2.imwrite(path,resized)           
    name = pic.split('\\')[1]         
    path = 'resized_test/'+pic.split('\\')[1]    
    label_data = find_plusten(resized,path,label_data,name)



In [389]:
label_data = pd.DataFrame(label_data,columns=['image_name', 'label_name','xmin','ymin','width','height','image_width','image_height'])
label_data

,image_name,label_name,xmin,ymin,width,height,image_width,image_height
0,M3-BH3299-1.jpg,wood,387,122,10,26,195,440
1,M3-BH3299-1.jpg,wood,300,82,8,26,195,440
2,M3-BH3299-1.jpg,+10cm rock,7,72,82,13,195,440
3,M3-BH3299-1.jpg,+10cm rock,123,71,61,12,195,440
4,M3-BH3299-2.jpg,wood,156,174,11,21,195,440
...,...,...,...,...,...,...,...,...
861,M3-BH3301-21.jpg,+10cm rock,293,66,143,15,195,440
862,M3-BH3301-21.jpg,+10cm rock,210,66,85,15,195,440
863,M3-BH3301-21.jpg,+10cm rock,248,39,55,10,195,440
864,M3-BH3301-21.jpg,+10cm rock,18,6,49,17,195,440


In [276]:
label_data.to_csv('label_test.csv')

In [277]:
get_rocks_length_sum('M3-BH3300-1.jpg')

[67.0, 31.5, 27.25]

In [412]:
def find_plusten(image,path,label_data,name):
    
    mask1 = cv2.inRange(image,(50,50,50), (90,90,90))
    mask2 = cv2.inRange(image, (110,115,125), (125,135,155))

    mask = cv2.bitwise_or(mask1, mask2 )
    mask = cv2.bitwise_not(mask)
#     image = cv2.bitwise_and(image,image,mask = mask)
    
    contours, hierarchy = cv2.findContours(image=mask, mode=cv2.RETR_TREE  , method=cv2.CHAIN_APPROX_NONE  )

    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)
    #     cv2.rectangle(result, (x,y), (x+w,y+h), (0, 255, 0), 2)

        if 9<h<45 and w>35:
            cv2.rectangle(image, (x,y), (x+w,y+h), (0, 255, 0), 2)
            label_data.append([path.split('/')[1], '+10cm rock',x,y,w,h,image.shape[0],image.shape[1]])
    cv2.imwrite(path,image)
    
    return label_data


In [208]:
def get_rocks_length_sum(image_name):
    pix_to_cm = 110/440
    woods = label_data[(label_data['label_name']=='wood') & (label_data['image_name']==image_name)].sort_values('ymin')
    rocks = label_data[(label_data['label_name']=='+10cm rock') & (label_data['image_name']==image_name)].sort_values('ymin')
    start_point = [0,0]
    end_point = [440,175]
    points_list =[]
    points_list.append(start_point)
    for x, y in zip(woods['xmin'], woods['ymin']):
        points_list.append([x,y])
    points_list.append(end_point)
    
    runs_length_list=[]
    for i in range(len(points_list)-1):
        run_length=0
        start = points_list[i]
        end = points_list[i+1]

        for x,y,w in zip(rocks['xmin'],rocks['ymin'],rocks['width']):
            if check_inrange(x,y,start,end):
                run_length +=(w*pix_to_cm)
#                 print(x,y,w*pix_to_cm,' in ', start,end)

        runs_length_list.append(run_length)
        
    return runs_length_list

In [123]:
def check_inrange(x,y,start,end):
        height = end[1] - start[1] 
        epsilon = 20
#     part one in the same row of start but right of it
        if (x - start[0])>0 and abs(y-start[1])<20:
            return True
#     part two between start and end in different rows
        elif  epsilon<(y - start[1]) < height:
            return True
#     part three in the same row of end but left of it
        elif (x- end[0])<0 and abs(y-end[1])<20:
            return True
        else:
            return False


In [196]:
def get_RQD(x):
    if 0<=x<=0.25:
        return 1
    if 0.25<x<=0.5:
        return 2
    if 0.5<x<=0.75:
        return 3
    if 0.75<x<=0.9:
        return 4
    if 0.9<x:
        return 5
    

In [390]:
runs_length_list={}
for pic in pics_list:
    image_name = pic.split('\\')[1]
    runs_length_list[image_name] =get_rocks_length_sum(image_name ) 
runs_length_list

{'M3-BH3299-1.jpg': [35.75, 0, 0],
 'M3-BH3299-2.jpg': [0, 0, 0, 14.75, 0, 0],
 'M3-BH3299-3.jpg': [32.25, 108.75, 94.5, 50.0, 158.0, 9.25],
 'M3-BH3299-4.jpg': [72.25, 138.5, 146.25, 20.75],
 'M3-BH3299-5.jpg': [9.5, 9.5, 180.5, 118.5, 20.25],
 'M3-BH3299-6.jpg': [13.25, 68.0, 0, 0],
 'M3-BH3299-7.jpg': [52.25, 104.75, 65.75, 32.5, 0],
 'M3-BH3299-8.jpg': [0, 33.5, 94.0, 121.0, 48.0],
 'M3-BH3299-9.jpg': [314.0, 111.75, 54.75, 35.75],
 'M3-BH3300-1.jpg': [0, 42.25, 10.0],
 'M3-BH3300-2.jpg': [91.75, 21.25],
 'M3-BH3300-3.jpg': [101.5, 0, 61.25, 41.25, 64.5, 35.5],
 'M3-BH3300-4.jpg': [107.75, 99.25, 174.75, 149.75, 81.25],
 'M3-BH3300-5.jpg': [33.0, 180.5, 249.0, 53.5],
 'M3-BH3300-6.jpg': [319.5, 279.5, 217.25],
 'M3-BH3300-7.jpg': [97.25, 247.5, 169.75, 150.5, 94.5],
 'M3-BH3300-8.jpg': [431.25, 67.25, 52.75],
 'M3-BH3300-9.jpg': [122.5, 447.25, 45.0],
 'M3-BH3300-10.jpg': [255.75, 206.5],
 'M3-BH3300-11.jpg': [99.75, 429.75, 56.75],
 'M3-BH3300-12.jpg': [170.5, 128.0, 74.5, 48.0],


In [391]:
runs_data = pd.read_excel('from-to-rqd.xlsx')
runs_data['length'] = (runs_data['to'] - runs_data['from'])*100

output_data =[]
for image in runs_data['RunId']:
     try:
#         print(image)
        output_data.append([image, runs_length_list[image[:-2]+'.jpg'][int(image[-1])-1]   ])
     except:
        output_data.append([image, 0 ])
output_data = pd.DataFrame(output_data,columns=['RunId','Prediction'])
output_data['Prediction'] = output_data['Prediction']/runs_data['length']
output_data['Prediction']= output_data['Prediction'].apply(lambda x:get_RQD(x)) 

output_data.to_csv('output.csv' , index=False)
output_data

,RunId,Prediction
0,M3-BH3299-1-1,1
1,M3-BH3299-1-2,1
2,M3-BH3299-1-3,1
3,M3-BH3299-2-1,1
4,M3-BH3299-2-2,1
...,...,...
175,M3-BH3301-20-1,5
176,M3-BH3301-20-2,1
177,M3-BH3301-20-3,5
178,M3-BH3301-20-4,3


In [311]:

output_data.to_csv('output.csv' , index=False)

In [383]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
%matplotlib auto

image = cv2.imread('resized2.jpg')
image = image[10:-10,:]

cv2.imshow('imj',image)
# Pink mask
mask1 = cv2.inRange(image,(50,50,50), (90,90,90))
mask2 = cv2.inRange(image, (110,110,70), (199,168,150))

## Merge the mask and crop the red regions
mask = cv2.bitwise_or(mask1, mask2 )
mask = cv2.bitwise_not(mask)
image = cv2.bitwise_and(image,image,mask = mask)


img=cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
edges = cv2.Canny(img,img.shape[0],img.shape[1])
cv2.imshow('edges',edges)
contours, hierarchy = cv2.findContours(image=mask, mode=cv2.RETR_TREE  , method=cv2.CHAIN_APPROX_NONE)

for contour in contours:
    (x,y,w,h) = cv2.boundingRect(contour)
#     cv2.rectangle(image, (x,y), (x+w,y+h), (0, 255, 0), 2)

    if 9<h<45 and w>35:
        cv2.rectangle(image, (x,y), (x+w,y+h), (0, 255, 0), 2)
#         label_data.append([path.split('/')[1], '+10cm rock',x,y,w,h,image.shape[0],image.shape[1]])
        
cv2.imshow('isssmj',image)

Using matplotlib backend: Qt5Agg
